# Custom Example #1

## <span style="color:purple">Exploratory data analysis</span>

In [1]:
from bokeh.io import output_notebook, show
from bokeh.plotting import Figure
from bokeh.layouts import layout #general and most flexible bokeh layout
output_notebook() # alternatively one could use output_file(file.html)

Loading BokehJS ...

In [2]:
import numpy as np
from bokeh.models import CDSView, ColumnDataSource as CDS, CustomJSFilter, Range1d

#generate the data
npoints = 10000
data = dict(exp=np.random.exponential(1, size=npoints),
            gau=np.random.normal(0, 0.5, size=npoints),
            exp2=np.random.exponential(7, size=npoints),
            gau2=np.random.normal(1, 2.5, size=npoints),
            )

source = CDS(data=data)

In [3]:
from bokeh.models import CustomJS, RangeSlider

NAMES = ['exp','gau','exp2','gau2']
RANGES_LOW = {NAMES[0]: 0, NAMES[1]: -3.5, NAMES[2]: 0, NAMES[3]: -15}
RANGES_HIGH = {NAMES[0]: 10, NAMES[1]: 3.5, NAMES[2]: 70, NAMES[3]: 15}

slider_exp  = RangeSlider(start=RANGES_LOW[NAMES[0]], end=RANGES_HIGH[NAMES[0]], 
                          value=(RANGES_LOW[NAMES[0]],RANGES_HIGH[NAMES[0]]), step=.1, title="Exp")

slider_gau  = RangeSlider(start=RANGES_LOW[NAMES[1]], end=RANGES_HIGH[NAMES[1]], 
                          value=(RANGES_LOW[NAMES[1]],RANGES_HIGH[NAMES[1]]), step=.1, title="Gau")

slider_exp2  = RangeSlider(start=RANGES_LOW[NAMES[2]], end=RANGES_HIGH[NAMES[2]], 
                           value=(RANGES_LOW[NAMES[2]],RANGES_HIGH[NAMES[2]]), step=.1, title="Exp2")

slider_gau2  = RangeSlider(start=RANGES_LOW[NAMES[3]], end=RANGES_HIGH[NAMES[3]], 
                           value=(RANGES_LOW[NAMES[3]],RANGES_HIGH[NAMES[3]]), step=.1, title="Gau2")

In [4]:
callback = CustomJS(args=dict(s=source), code="""
    s.change.emit();
""")

slider_exp.js_on_change('value_throttled', callback)
slider_gau.js_on_change('value_throttled', callback)
slider_exp2.js_on_change('value_throttled', callback)
slider_gau2.js_on_change('value_throttled', callback)

In [5]:
def get_custom_filter(slider, variable):
    return CustomJSFilter(args=dict(slider=slider), code="""
        var indices = [];
        var start = slider.value[0];
        var end = slider.value[1];

        const data = source.data['{var}'];

        for (var i=0; i < source.get_length(); i++){{
            indices[i] = data[i] >= start && data[i] <= end;
        }}
        return indices;
        """.format(var=variable))

fexp  = get_custom_filter(slider_exp,  NAMES[0])
fgau  = get_custom_filter(slider_gau,  NAMES[1])
fexp2 = get_custom_filter(slider_exp2, NAMES[2])
fgau2 = get_custom_filter(slider_gau2, NAMES[3])

In [6]:
# Use the filter in a view
view = CDSView(source=source, 
               filters=[fexp, fgau, fexp2, fgau2])

Explore more markers [here](https://docs.bokeh.org/en/latest/docs/reference/plotting.html)

In [7]:
COLORS = ['brown', 'purple', 'orange', 'green']
MARKERS = ['triangle', 'square', 'square_pin', 'diamond']
XVAR = ['exp', 'gau', 'exp', 'gau']
YVAR = ['exp2', 'exp2', 'gau2', 'gau2']

figs = []
for i in range(len(COLORS)):
    figs.append( Figure(plot_width=450, plot_height=350, output_backend='webgl') )
    figs[-1].scatter(x=XVAR[i], y=YVAR[i], 
                     color=COLORS[i], marker=MARKERS[i],
                     source=source, view=view)
    figs[-1].xaxis.axis_label = XVAR[i]
    figs[-1].yaxis.axis_label = YVAR[i]
    
    #fix ranges (otherwise they change during filtering)
    figs[-1].x_range=Range1d(RANGES_LOW[XVAR[i]]-1, RANGES_HIGH[XVAR[i]]+1)
    figs[-1].y_range=Range1d(RANGES_LOW[YVAR[i]]-1, RANGES_HIGH[YVAR[i]]+1)

In [8]:
lay = layout([[slider_gau],[slider_exp],[slider_exp2],[slider_gau2],
              [figs[0], figs[1]], 
              [figs[2], figs[3]]])
show(lay)

One could have instead a manual selection (hold the SHIFT key for multiple selections):

In [9]:
#add data to a ColumnDataSource
source.data['color'] = ['grey' for _ in range(npoints)]
print(source.data.keys())

figs2 = []
for i in range(len(COLORS)):
    figs2.append( Figure(plot_width=450, plot_height=350, output_backend="webgl") )
    figs2[-1].scatter(x=XVAR[i], y=YVAR[i], 
                      color='color', #now the scatter plot is also linked to the CDS color column 
                      marker=MARKERS[i],
                      source=source)
    figs2[-1].xaxis.axis_label = XVAR[i]
    figs2[-1].yaxis.axis_label = YVAR[i]

dict_keys(['exp', 'gau', 'exp2', 'gau2', 'color'])


In [14]:
source.selected.js_on_change('indices', CustomJS(args=dict(s1=source), code="""
        var inds = cb_obj.indices;
        var dcol = s1.data['color'];
        
        for (var i=0; i<inds.length; i++) {
            dcol[ inds[i] ] = 'red';
        }
        
        s1.change.emit();
        
        for (var i=0; i<inds.length; i++) {
            dcol[ inds[i] ] = 'grey';
        }
    """)
)

In [13]:
from bokeh.models import BoxSelectTool, ResetTool
for f in figs2:
    f.tools = [BoxSelectTool(dimensions='both'), ResetTool()]
lay2 = layout([[figs2[0], figs2[1]], 
              [figs2[2], figs2[3]]])
show(lay2)

One can alternatively obtain the same result with a different, more general [technique](https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html#customjs-for-tools).

See other available tools [here](https://docs.bokeh.org/en/latest/docs/user_guide/tools.html#configuring-plot-tools).